# STAC Item Search and Submission to Data Pipeline

This notebook allows operators to:
1. Define an area of interest (AOI) and time range
2. Search for STAC items from the EOPF STAC catalog
3. Submit selected items to the data pipeline for processing

## Setup and Imports

In [ ]:
import json
import os
from datetime import datetime
from typing import List, Dict, Optional

import pika
from pystac_client import Client
from shapely.geometry import box
import pandas as pd

## Configuration

In [ ]:
# STAC API Configuration
STAC_API_URL = "https://stac.core.eopf.eodc.eu/"

# RabbitMQ Configuration
RABBITMQ_HOST = "localhost"
RABBITMQ_PORT = 5672
RABBITMQ_VHOST = "/"
RABBITMQ_USER = "user"
RABBITMQ_PASSWORD = os.getenv("RABBITMQ_PASSWORD")
RABBITMQ_EXCHANGE = "eopf_samples"
RABBITMQ_ROUTING_KEY = "eopf_samples.convert"

if not RABBITMQ_PASSWORD:
    print("⚠️  Warning: RABBITMQ_PASSWORD environment variable not set!")
else:
    print("✅ RabbitMQ password loaded from environment")

## Define Area and Time of Interest

In [ ]:
# Area of Interest (AOI) - Bounding box: [min_lon, min_lat, max_lon, max_lat]
# Example: Vienna area
aoi_bbox = [16.2, 48.1, 16.5, 48.3]

# Time range
start_date = "2024-10-01T00:00:00Z"
end_date = "2024-10-31T23:59:59Z"

print(f"Area of Interest: {aoi_bbox}")
print(f"Time Range: {start_date} to {end_date}")

## Browse Available Collections

In [ ]:
# Connect to STAC API
catalog = Client.open(STAC_API_URL)

# List available collections
collections = list(catalog.get_collections())

print(f"\n📚 Available Collections ({len(collections)} total):\n")
for col in collections:
    print(f"  - {col.id}")
    if col.description:
        print(f"    {col.description[:100]}..." if len(col.description) > 100 else f"    {col.description}")
    print()

## Select Collection and Search for Items

In [ ]:
# Choose the source collection to search
source_collection = "sentinel-2-l2a"  # Change this to your desired collection

# Choose the target collection for processing
target_collection = "sentinel-2-l2a-dp-test"  # Change this to your target collection

print(f"🔍 Searching collection: {source_collection}")
print(f"🎯 Target collection for processing: {target_collection}")

In [ ]:
# Search for items
search = catalog.search(
    collections=[source_collection],
    bbox=aoi_bbox,
    datetime=f"{start_date}/{end_date}",
    max_items=100  # Adjust as needed
)

# Collect items
items = list(search.items())

print(f"\n✅ Found {len(items)} items")

# Display items in a table
if items:
    items_data = []
    for item in items:
        items_data.append({
            "ID": item.id,
            "Collection": item.collection_id,
            "Datetime": item.datetime.isoformat() if item.datetime else "N/A",
            "Self Link": next((link.href for link in item.links if link.rel == "self"), "N/A")
        })
    
    df = pd.DataFrame(items_data)
    display(df)
else:
    print("No items found for the specified criteria.")

## Submit Items to Pipeline

In [ ]:
def submit_item_to_pipeline(item_url: str, target_collection: str) -> bool:
    """
    Submit a single STAC item to the data pipeline via RabbitMQ.
    
    Args:
        item_url: The self-link URL of the STAC item
        target_collection: The target collection for processing
    
    Returns:
        True if successful, False otherwise
    """
    try:
        # Create payload
        payload = {
            "source_url": item_url,
            "collection": target_collection,
        }
        
        # Connect to RabbitMQ
        credentials = pika.PlainCredentials(RABBITMQ_USER, RABBITMQ_PASSWORD)
        connection = pika.BlockingConnection(
            pika.ConnectionParameters(
                RABBITMQ_HOST,
                RABBITMQ_PORT,
                RABBITMQ_VHOST,
                credentials
            )
        )
        channel = connection.channel()
        
        # Publish message
        message = json.dumps(payload)
        channel.basic_publish(
            exchange=RABBITMQ_EXCHANGE,
            routing_key=RABBITMQ_ROUTING_KEY,
            body=message,
            properties=pika.BasicProperties(content_type="application/json"),
        )
        
        connection.close()
        return True
        
    except Exception as e:
        print(f"❌ Error submitting item: {e}")
        return False

In [ ]:
# Submit all found items to the pipeline
if items:
    print(f"\n📤 Submitting {len(items)} items to pipeline...\n")
    
    success_count = 0
    fail_count = 0
    
    for item in items:
        # Get the self link (canonical URL for the item)
        item_url = next((link.href for link in item.links if link.rel == "self"), None)
        
        if not item_url:
            print(f"⚠️  Skipping {item.id}: No self link found")
            fail_count += 1
            continue
        
        # Submit to pipeline
        if submit_item_to_pipeline(item_url, target_collection):
            print(f"✅ Submitted: {item.id}")
            success_count += 1
        else:
            print(f"❌ Failed: {item.id}")
            fail_count += 1
    
    print(f"\n📊 Summary:")
    print(f"  - Successfully submitted: {success_count}")
    print(f"  - Failed: {fail_count}")
    print(f"  - Total: {len(items)}")
else:
    print("No items to submit.")

## Submit Specific Items (Optional)

If you want to submit only specific items instead of all found items, you can manually select them:

In [ ]:
# Example: Submit only specific items by index
# Uncomment and modify as needed

# selected_indices = [0, 1, 2]  # Select first 3 items
# 
# for idx in selected_indices:
#     if idx < len(items):
#         item = items[idx]
#         item_url = next((link.href for link in item.links if link.rel == "self"), None)
#         
#         if item_url:
#             if submit_item_to_pipeline(item_url, target_collection):
#                 print(f"✅ Submitted: {item.id}")
#             else:
#                 print(f"❌ Failed: {item.id}")
#     else:
#         print(f"⚠️  Index {idx} out of range")